<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-07-26 22:18:30--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  30.5MB/s    in 8.3s    

2020-07-26 22:18:39 (30.2 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [70]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import applications
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.models import load_model

In [11]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

train_generator = data_generator.flow_from_directory('concrete_data_week4/train',target_size=(image_resize, image_resize),batch_size=batch_size_training,class_mode='categorical')
validation_generator = data_generator.flow_from_directory('concrete_data_week4/valid',target_size=(image_resize, image_resize),batch_size=batch_size_validation,class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [12]:
model = Sequential()
model.add(vgg16.VGG16(include_top=False, pooling='avg',weights='imagenet',))

58892288/58889256 [==============================] - 2s 0us/step


In [13]:
model.add(Dense(num_classes, activation='softmax'))

In [14]:
model.layers[0].trainable = False

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [17]:
fit_history = model.fit_generator(train_generator,steps_per_epoch=steps_per_epoch_training,epochs=num_epochs,validation_data=validation_generator,
              validation_steps=steps_per_epoch_validation,verbose=1,)

Epoch 1/2
301/301 [==============================] - 111s 368ms/step - loss: 0.0735 - accuracy: 0.9779 - val_loss: 0.0197 - val_accuracy: 0.9951
Epoch 2/2
301/301 [==============================] - 102s 338ms/step - loss: 0.0172 - accuracy: 0.9957 - val_loss: 0.0000e+00 - val_accuracy: 0.9969


In [18]:
model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

# **This is the evaluation of VGG16, I create the Image Generator and then Evaluate.**

In [19]:
directory = "/content/concrete_data_week4/test"
data_gen = ImageDataGenerator ()
image_gen = data_gen.flow_from_directory (directory, batch_size = 100, shuffle = False)


Found 500 images belonging to 2 classes.


In [31]:
scoreVGG = model.evaluate_generator(image_gen, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

In [34]:
print("Accuracy of VGG16 = ",scoreVGG[1])

Accuracy of VGG16 =  0.9419999718666077


# **Here I import ResNet50 and Evaluate on the same Image Generator.**

In [71]:
model2 = load_model('/content/classifier_resnet_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [72]:
scoreRES = model2.evaluate_generator(image_gen, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

In [73]:
print("Accuracy of ResNet50 = ",scoreRES[1])

Accuracy of ResNet50 =  0.6140000224113464


# **VGG16 Accuracy = 94%**
# **ResNet50 Accuracy = 61%**

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [35]:
predictVGG = model.predict_generator(image_gen, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

In [75]:
predictRES = model2.predict_generator(image_gen, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

**Prediction List for VGG16**

In [66]:
resultVGG = []
for i in range(5):
  argmaxVGG = np.argmax (predictVGG[i])
  if argmaxVGG == 0:
    resultVGG.append("Negative")
  else:
    resultVGG.append("Positive")
  print (resultVGG[i], "\n")



Negative 

Negative 

Negative 

Negative 

Negative 



**Prediction List for ResNet50**

In [77]:
resultRES = []
for i in range(5):
  argmaxRES = np.argmax (predictRES[i])
  if argmaxRES == 0:
    resultRES.append("Negative")
  else:
    resultRES.append("Positive")
  print (resultRES[i], "\n")

Negative 

Negative 

Negative 

Negative 

Negative 



EXTRA : Other prediction results down the list for bot VGG16 and ResNet50.

In [97]:
resultVGG = []
resultRES = []
j = 0
k = 0
print ("\n","VGG16")
for i in range(20,500,80):
  argmaxVGG = np.argmax (predictVGG[i])
  if argmaxVGG == 0:
    resultVGG.append("Negative")
  else:
    resultVGG.append("Positive")
  print (resultVGG[j], "\n")
  j += 1

print ("\n","RESNET50")
for i in range(20,500,80):
  argmaxRES = np.argmax (predictRES[i])
  if argmaxRES == 0:
    resultRES.append("Negative")
  else:
    resultRES.append("Positive")
  
  print (resultRES[k], "\n")
  k += 1


 VGG16
Negative 

Negative 

Negative 

Positive 

Positive 

Positive 


 RESNET50
Negative 

Negative 

Negative 

Negative 

Negative 

Negative 



### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).